# Named-entity recognition (NER)

- using Bi-LSTM

# 1)- Importing key modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [0]:
# For data processing and maths
import numpy as np
import pandas as pd
import time
import math
import os
#For Visuals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [3]:
! pip install jdc

In [4]:
# for deep learning and neural network
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import jdc
import json
from collections import defaultdict, OrderedDict
import math
import numpy as np

torch.manual_seed(1)

In [5]:
! pip install version_information

  Created wheel for version-information: filename=version_information-1.0.3-cp36-none-any.whl size=3880 sha256=a1e58195737ace4f7cbf73752a29ce4dd9416100e1cc5567a222e17be383224c
  Stored in directory: /root/.cache/pip/wheels/1f/4c/b3/1976ac11dbd802723b564de1acaa453a72c36c95827e576321
Successfully built version-information


In [6]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,torch,numpy,seaborn, matplotlib

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.24.2
torch 1.3.0+cu100
numpy 1.16.5
seaborn 0.9.0
matplotlib 3.0.3
Tue Oct 22 16:52:39 2019 UTC

In [7]:
# check device if it is gpu or cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# 2)- Example data & its preprocessing

### 2.1)- Helper Function

In [0]:
def argmax(vec):
    """Return the argmax as a python int"""
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    """
    Input:
        seq - the sequence (array)
        to_ix - the indices to which seqence values are converted (dict)

    Output:
        Numerical tensor
        """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def log_sum_exp(vec):
    """Compute log sum exp in a numerically stable way for 
    the forward algorithm"""
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

### 2.2)- Getting doccano

https://github.com/chakki-works/doccano#installation

In [39]:
#! pip install psycopg2 

In [40]:
#! git clone https://github.com/chakki-works/doccano.git


Cloning into 'doccano'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 7243 (delta 99), reused 113 (delta 59), pack-reused 7075
Receiving objects: 100% (7243/7243), 41.89 MiB | 29.10 MiB/s, done.
Resolving deltas: 100% (5049/5049), done.


In [41]:
#cd doccano

/content/doccano/app/server/doccano


In [42]:
#! sudo apt-get install libpq-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libpq-dev is already the newest version (10.10-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


In [43]:
#! pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/18/ab/133c68bbea94839d8f3b8b4aea4f70e1c6b8ac929aba4adbadc458566a76/django_pyodbc_azure-2.1.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0e/77/b2eef0c31c8c623ef73fcf6cd3c1c7f22d7ee9bbbb0c6ba36ca5a3c9f07a/django_rest_polymorphic-0.1.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/90/30/ad1148098ff0c375df2a30cc4494ed953cf7551fc1ecec30fc951c712d20/djangorestframework-3.8.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c1/f9/12c26113312212b0869ecabdcf3bfb3c14088a2f4fe2107c3a9975f78a8b/djangorestframework_filters-0.10.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1a/cf/266e7875e981a8096500bd8b4858816d482715a7579f3e6501504f036635/environs-4.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/29/321c2be2995d28d0daf7c91093520b00ed2c7ac1bd5a5b5a744eed76214a/djangorestframework_xml-1.4.0-py2

In [44]:
#cd app

/content/doccano/app/server/doccano/app


In [45]:
#cd server/static

/content/doccano/app/server/doccano/app/server/static


In [46]:
#! npm install


> husky@0.13.4 install /content/doccano/app/server/doccano/app/server/static/node_modules/husky
> node ./bin/install.js

husky
setting up hooks
done


> swiper@4.3.3 postinstall /content/doccano/app/server/doccano/app/server/static/node_modules/swiper
> node -e "console.log('\u001b\u001bLove Swiper? Support Vladimir\'s work by donating or pledging on patreon:\u001b\u001b\n > \u001bhttps://patreon.com/vladimirkharlampidi\u001b\n')"

Love Swiper? Support Vladimir's work by donating or pledging on patreon:
 > https://patreon.com/vladimirkharlampidi

npm WARN server@1.0.0 No description
npm WARN server@1.0.0 No repository field.
npm WARN optional SKIPPING OPTIONAL DEPENDENCY: fsevents@1.2.9 (node_modules/fsevents):
npm WARN notsup SKIPPING OPTIONAL DEPENDENCY: Unsupported platform for fsevents@1.2.9: wanted {"os":"darwin","arch":"any"} (current: {"os":"linux","arch":"x64"})

added 966 packages from 766 contributors in 19.489s


In [47]:
#! npm run build


> server@1.0.0 build /content/doccano/app/server/doccano/app/server/static
> webpack

Hash: 85ff092dbdfb9c9f223a
Version: webpack 4.12.0
Time: 6745ms
Built at: 2019-10-22 16:50:41
                          Asset      Size                        Chunks             Chunk Names
                       index.js   683 KiB                         index  [emitted]  index
                     dataset.js   147 KiB                       dataset  [emitted]  dataset
    demo_text_classification.js  1.42 MiB      demo_text_classification  [emitted]  demo_text_classification
            demo_translation.js   1.4 MiB              demo_translation  [emitted]  demo_translation
     document_classification.js  1.33 MiB       document_classification  [emitted]  document_classification
            download_seq2seq.js  1.04 MiB              download_seq2seq  [emitted]  download_seq2seq
  download_sequence_labeling.js  1.04 MiB    download_sequence_labeling  [emitted]  download_sequence_labeling
download_te

In [48]:
#cd ..

/content/doccano/app/server/doccano/app/server


# 3)- Loading Data

In [9]:
# Read export file
with open('doccano_export.json', 'r') as f:
    lines = f.readlines()

print(json.loads(lines[1]))

{'id': 35, 'text': "Now, I'm ready to move this to a serving environment (via Sagemaker, but that just implements tensorflow.serving).", 'annotations': [{'label': 4, 'start_offset': 0, 'end_offset': 3, 'user': 1}, {'label': 4, 'start_offset': 5, 'end_offset': 8, 'user': 1}, {'label': 4, 'start_offset': 9, 'end_offset': 14, 'user': 1}, {'label': 4, 'start_offset': 15, 'end_offset': 17, 'user': 1}, {'label': 4, 'start_offset': 18, 'end_offset': 22, 'user': 1}, {'label': 4, 'start_offset': 23, 'end_offset': 27, 'user': 1}, {'label': 4, 'start_offset': 28, 'end_offset': 30, 'user': 1}, {'label': 4, 'start_offset': 31, 'end_offset': 32, 'user': 1}, {'label': 4, 'start_offset': 33, 'end_offset': 40, 'user': 1}, {'label': 4, 'start_offset': 41, 'end_offset': 52, 'user': 1}, {'label': 4, 'start_offset': 54, 'end_offset': 57, 'user': 1}, {'label': 4, 'start_offset': 58, 'end_offset': 67, 'user': 1}, {'label': 4, 'start_offset': 69, 'end_offset': 72, 'user': 1}, {'label': 4, 'start_offset': 73, 

### 3.1)-Split out words and labels

In [0]:
# The numerical doccano label to actual label (B-I-O scheme)
ix_to_label = {4: 'O', 3: 'I', 2: 'B'}
    
# train/test data
data = []

# Vocabulary
vocab = set()
    
# Loop over each data point (a corpus of labeled text) to extract words
for line in lines:
    # An ordered dict will keep items in order for further manipulation
    # so we initialize here
    orddict = OrderedDict({})
    # Lists to hold the words and labels
    words = []
    labels = []
    # Convert line to json
    injson = json.loads(line)
    annots = injson['annotations']
    text = injson['text']
    
    # Add each word annotation to OrderedDict
    for ann in annots:
        orddict[ann['start_offset']] = ann
    
    # Sort ordered dict because there's no guarantee reading json
    # maintained order
    orddict = sorted(orddict.items(), key=lambda x: x[1]['start_offset'])
    
    for item in orddict:
        # the item is a tuple where second value is the actual value we want
        ann = item[1]
        # Subset text string
        word = text[ann['start_offset']:(ann['end_offset'] + 1)].rstrip()
        label = ix_to_label[ann['label']]
        # Add to list for this datum/corpus
        words.append(word)
        labels.append(label)
        vocab.add(word)
    # Add to overall data containers
    data.append((words, labels))

### 3.2)-Split into train and test data

In [0]:
num_train = math.floor(len(data) * 0.8) # 80% to train
training_data, test_data = data[:num_train], data[num_train:]

# 4)- Building model

### 4.1)- Tags and Hyper-parameter

In [0]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
MINIBATCH_SIZE = 2
LEARNING_WEIGHT = 5e-2
WEIGHT_DECAY = 1e-4

### 4.2)- CRF Class

In [0]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        """Initialize network."""
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

### 4.3)- Function to initialize hidden state

In [0]:
%%add_to BiLSTM_CRF

def init_hidden(self):
    """Two tensors to hold hidden states, one for each
    LSTM direction with dimensions of (num_layers, 
    minibatch, hidden_dim)"""
    # Minibatch small because small dataset below
    return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
            torch.randn(2, 1, self.hidden_dim // 2).to(device))

### 4.4)- Conditional Random Field

In this case, iteration has been made per sentence i.e one sentence is one training data point.For each sentence, there is multiple embeddings

In [0]:
%%add_to BiLSTM_CRF


def _forward_alg(self, feats):
    """Core magic of the Conditional Random Field.  
    
    Input:
        The word embeddeding vectors for a sentence
    
    Since we’re using PyTorch to compute gradients for us, 
    we technically only need the forward part of the forward-backward 
    algorithm """
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
    # START_TAG ("<START>") has all of the score.
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(self.tagset_size):
            # broadcast the emission score: it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(
                1, -1).expand(1, self.tagset_size)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = self.transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

### 4.5)- For LSTM features

In [0]:
%%add_to BiLSTM_CRF

def _get_lstm_features(self, sentence):
    """Compute output vector of BiLSTM - used in 
    the forward pass of network"""
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    # Map LSTM features into tag space
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

### 4.6)- Score

In [0]:
%%add_to BiLSTM_CRF

def _score_sentence(self, feats, tags):
    """Gives the score of a provided tag sequence"""
    # Gives the score of a provided tag sequence
    score = torch.zeros(1).to(device)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), 
                      tags])
    for i, feat in enumerate(feats):
        score = score + \
            self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

### 4.7)- Viterbi

In [0]:
%%add_to BiLSTM_CRF

def _viterbi_decode(self, feats):
    """Implements Viterbi algorithm for finding most likely sequence of labels.
    Used in the forward pass of the network.

    We take the maximum over the previous states as opposed to the sum. 
    Input:
        loglikelihoods: torch tensor.
    Output:
        tuple. The first entry is the loglikelihood of this sequence. The second is 
        the most likely sequence of labels. 
    """
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  # holds the backpointers for this step
        viterbivars_t = []  # holds the viterbi variables for this step

        for next_tag in range(self.tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the
            # previous step, plus the score of transitioning
            # from tag i to next_tag.
            # We don't include the emission scores here because the max
            # does not depend on them (we add them in below)
            next_tag_var = forward_var + self.transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1).to(device)
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path


### 4.8)- Negative Log Likelihood

In [0]:
%%add_to BiLSTM_CRF

def neg_log_likelihood(self, sentence, tags):
    """Calculate the negative log likelihood given a sequence and labels.
    This is used in training (only) because we don't need to create
    and check the B-I-O tags themselves - only the score is important
    here for calculating the loss."""
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_alg(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

### 4.9)- Forward Pass

In [0]:
%%add_to BiLSTM_CRF

def forward(self, sentence):
    """The forward pass function for training the network.
    This is used in inference only."""
    # Get the emission scores (output layer) from the 
    # BiLSTM 
    lstm_feats = self._get_lstm_features(sentence)

    # Find the best path, given the features.
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

**End of our class. %%add is adding whole story from one cell to other**

### 4.10)- Removing punctuation from text

In [0]:
def remove_punct(text):
    """Remove punctuation from a piece of text"""
    punct = list(".,()-")
    for p in punct:
        text = text.replace(p, '')
    return text
    
text = remove_punct(text)

### 4.11)-lookup dictionary

In [0]:
# Create a lookup dict for all possible words and record their index
word_to_ix = {k: v for (k, v) in zip(vocab, range(len(vocab)))}
tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
ix_to_tag = {0: "B", 1: "I", 2: "O"}

### 4.12)-Initialize model and optimizer for training

In [0]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_WEIGHT, weight_decay=WEIGHT_DECAY)

In [25]:
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', training_data[0][1])
    print(training_data[0][0])

Prediction:    ['O', 'O', 'B', 'I', 'B', 'I', 'O', 'B', 'I', 'B', 'I', 'O', 'B', 'I', 'O', 'B', 'I', 'O', 'B', 'I', 'B', 'I', 'O', 'B', 'I', 'O', 'B', 'I', 'O', 'B', 'I', 'O', 'B', 'I', 'O', 'B', 'I', 'O', 'B', 'I', 'B', 'I', 'B', 'I', 'O', 'B', 'I']
Ground truth:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']
['This', 'makes', 'it', 'harder', 'to', 'understand', 'the', 'behavior', 'of', 'the', 'function', 'tf.scatter_add', 'in', 'case', 'indices', 'is', 'a', 'matrix.', 'Specifically,', 'what', 'is', 'the', 'difference', 'between', 'tf.scatter_add', 'and', 'tf.scatter_nd w', 'when', 'indices', 'is', 'a', 'matrix.', 'This', 'will', 'raise', 'an', 'error', 'that', 'only', 'sequential', 'or', 'functional', 'models', 'can', 'be', 'saved', "model.save('custom_model.h5')"]


Not a very good job at this point. Hopefully, we will get better results after training.

# 5)-Train model!

In [26]:
%%time
# Make sure prepare_sequence from earlier in the LSTM section is loaded
# again, normally you would do more than 300 epochs, but we have
# toy data

losses = []
for epoch in range(100):  
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance of LSTM
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        sentence_in, targets = sentence_in.to(device), targets.to(device)

        # Step 3. A lot happens.  Run our forward pass to get features from BLSTM,
        # run the CRF and get the negative log likelihoods and find the best 
        # "path" through sentence with the tags using the viterbi algorithm 
        # (also part of forward pass).
        # BTW our dynamic computational graph is created with the forward pass
        # Returns the forward score - ground truth score (our loss measure)
        loss = model.neg_log_likelihood(sentence_in, targets)
        losses.append(loss.item())


        # Step 4. Compute the loss, gradients (backprop), and update the 
        # parameters by calling optimizer.step() - optimizer here is 
        # SGD for our CRF
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print("Epoch: {} Loss: {}".format(epoch+1, np.mean(losses)))

Epoch: 10 Loss: 9.635039234161377
Epoch: 20 Loss: 8.520493650436402
Epoch: 30 Loss: 7.006024519602458
Epoch: 40 Loss: 5.711645150184632
Epoch: 50 Loss: 4.711056823730469
Epoch: 60 Loss: 3.9849269866943358
Epoch: 70 Loss: 3.4500631604875838
Epoch: 80 Loss: 3.033038914203644
Epoch: 90 Loss: 2.7100445641411675
Epoch: 100 Loss: 2.4464377975463867
CPU times: user 1min 34s, sys: 1.21 s, total: 1min 35s
Wall time: 1min 36s


### Save model


save weights not shape

In [0]:
torch.save(model.state_dict(), 'model_0.pth')

In [29]:
model.eval()

BiLSTM_CRF(
  (word_embeds): Embedding(182, 5)
  (lstm): LSTM(5, 2, bidirectional=True)
  (hidden2tag): Linear(in_features=4, out_features=5, bias=True)
)

In [28]:
# Sanity check for predictions after training
# No need to accumulate gradients because this is a validation
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[1][0], word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', training_data[1][1])
    print(training_data[1][0])

Prediction:    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']
Ground truth:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B']
['Now,', "I'm", 'ready', 'to', 'move', 'this', 'to', 'a', 'serving', 'environment', 'via', 'Sagemaker,', 'but', 'that', 'just', 'implements', 'tensorflow.serving)']


with *torch.no_grad()* is used so that gradients are not propagated which is important for when we are running predictions and scoring

# 6)-Evaluate

Let's test our model on an unseen sentence.

In [30]:
model.eval() # set again just in case

# Pick some test data
test_datum = test_data[0][0]
test_text = test_data[0][1]

with torch.no_grad():
    precheck_sent = prepare_sequence(test_datum, word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', test_text)
    print('Text: ', test_datum)

Prediction:    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Ground truth:  ['O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Text:  ['I', 'have', 'a', 'tf.data.Dataset', 'that', 'I', 'want', 'to', 'write', 'to', 'tfrecord', 'files.', 'to', 'do', 'this,', 'I', 'currently', 'use', 'tf.python_io.TFRecordWriter', 'to', 'do', 'this,', 'but', 'would', 'like', 'to', 'use', 'tf.data.experimental.TFRecordWriter,', 'as', 'it', 'would', 'be', 'more', 'efficient', 'to', 'also', 'do', 'the', 'writing', 'as', 'part', 'of', 'the', 'dataset', 'graph', 'execution.']
